In [10]:
from collections import namedtuple


Point = namedtuple('Point',['window', 'snp', 'bp'])

class Node:
    def __init__(self, start, end):
        self.start = start #Point()
        self.end = end #Point()
    
    def __repr__(self):
        return ' '.join([str(self.start), str(self.end)])

In [11]:
a = Point(20, 0, 15)
b = Point(20, 14, 18)
Node(a, b)

Point(window=20, snp=0, bp=15) Point(window=20, snp=14, bp=18)

In [ ]:
from pandas import DataFrame, read_csv
# read_csv()
import os

def read_data(file_path = "../test_data/KE_chromo10.txt"):
    """Individuals are rows, not columns"""
    loci = []
    with open(file_path) as ke:
        for line in ke.readlines():
            loci.append(tuple(int(x) for x in line.split()))
    return loci
alleles = read_data()
print(alleles[1], len(alleles))
print(alleles[-1])

In [46]:
import numpy as np
individuals = np.array(alleles).T.tolist()
assert len(individuals[1]) == 32767
assert len(individuals) == 501

(32767, 501)

In [53]:
def get_unique_signatures(individuals, start_locus, block_size = 20):
    unique_blocks = set()
    for individual in individuals:
        unique_blocks.add(tuple(individual[start_locus : start_locus + block_size]))
    return unique_blocks
unique_blocks = get_unique_signatures(individuals, 0 )
    
assert len(unique_blocks) == 4
assert unique_blocks == {(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2),
 (0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0),
 (2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2)}

for loci_start in 